In [1]:
import numpy as np
import math

filename = 'SineTestAngle.gcode'

with open(filename, 'w') as file:
    file.write('; This code was generated by Leviticus Rhodens Non-planar code.\n')


In [2]:
def addPoint(x,y,z,e,f):
    '''This appends a new co-ordinate to the gcode file! It takes in x,y,z,and e co-ordinates as floats.'''
    #set zero as center of buildplate
    x += 110
    y += 110
    with open(filename,'a') as file:
        if z<0:
            print("ERROR: I'm under the print bed!")
            return
        if e==0:
            file.write('G0 ')
        else:
            file.write('G1 ')
        file.write('F' + "{:#.6g}".format(f) + ' X' + "{:#.6g}".format(x) + ' Y' + "{:#.6g}".format(y) + ' Z' + "{:#.6g}".format(z) + ' E' + "{:#.6g}".format(e) + '\n')
        
def addComment(note):
    '''Takes in a string note and adds it as a comment to the GCODE file, does not need to have the ; in front'''
    with open(filename,'a') as file:
        file.write(";" + note +"\n")


In [3]:
#these functions define the surface maps we are transitioning between
#just make sure that the map at each point (0, z2 and z3) dosn't overlap with the other maps.
def surface2(x,y):
    return 2*np.sin(3*math.atan2(y,x))
def surface3(x,y):
    return 0


def surface(x,y,z):
    '''This takes in the x,y, and planar z float, and returns the deltaZ from the planar z'''
    # README This is determined by YOU, and is able to change.
    
    if z<z2:
        dz = (z/z2)*surface2(x,y)
    else:
        dz = ((z-z2)/(z3-z2))*surface2(x,y)
    return dz

In [4]:
# Write the startup code

with open(filename,'a') as file:
    file.write('; Ender 3 Custom Start G-code\n')
    file.write('G92 E0 ; Reset Extruder\n')
    file.write('G28 ; Home all axes\n')
    file.write('G1 Z2.0 F3000 ; Move Z Axis up little to prevent scratching of Heat Bed\n')
    file.write('G1 X0.1 Y20 Z0.3 F5000.0 ; Move to start position\n')
    file.write('M140 S50 ;I think these next few lines are heating. I will know if it heats\nM105\nM190 S50\nM104 S200\nM105\nM109 S200')
    file.write('G1 X0.1 Y200.0 Z0.3 F1500.0 E15 ; Draw the first line\n')
    file.write('G1 X0.4 Y200.0 Z0.3 F5000.0 ; Move to side a little\n')
    file.write('G1 X0.4 Y20 Z0.3 F1500.0 E30 ; Draw the second line\n')
    file.write('G92 E0 ; Reset Extruder\n')
    file.write('G1 Z2.0 F3000 ; Move Z Axis up little to prevent scratching of Heat Bed\n')
    file.write('G1 X5 Y20 Z0.3 F5000.0 ; Move over to prevent blob squish\n')
    file.write('G90 ; make all co-ordinates absolute \nM83 ; make E relative \n')
    #file.write('G0 F1500\n')
    file.write(';TYPE SKIN\n;------------\n')
    

In [5]:
layerH = 0.2 #layer height in mm
width = 0.4 #for now, it's the nozzel width
radius = 10 #radius of corral in mm
dX = 0.5

switch1 = 6 #first pattern switch
z2 = switch1

Z=layerH
layer = 0
while Z < switch1:
    layerStr = "LAYER " + str(layer)
    addComment(layerStr)
    addComment('TYPE SKIN')
    for r in np.linspace(dX,radius,round(radius/width)):
        dTheta = dX/r
        for theta in np.linspace(0,2*3.14159,round(2*3.14159/dTheta)):
            X = r*np.sin(theta)
            Y = r*np.cos(theta)
            lastX = r*np.sin(theta-dTheta)
            lastY = r*np.cos(theta-dTheta)
            
            #This calculates volume of extrusion, then divides by cross area of filament
            #(GCODE takes it linear extrusion)
            dZ = surface(X,Y,Z) - surface(X,Y,Z-layerH)
            lastDZ = surface(lastX,lastY,Z) - surface(lastX,lastY,Z-layerH)
            vol = dX*width*(dZ + 0.5*(lastDZ-dZ) + layerH)
            e = vol/2.41
            
            addPoint(X,Y,Z+surface(X,Y,Z),e,1200)
        
    Z += layerH
    layer += 1
    
print('done!')

done!
